# `sc_linac` Live

In [1]:
import lcls_live
import os
os.__file__

'/Users/mpe/miniconda3/envs/lcls-live-dev/lib/python3.8/os.py'

In [2]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [3]:
from lcls_live.datamaps import get_datamaps
from lcls_live.archiver import lcls_archiver_restore

from lcls_live.tools import isotime

import matplotlib.pyplot as plt
import numpy as np

import os

In [4]:
# Pick a model and slice

MODEL = 'sc_bsyd' # or sc_diag0 or sc_inj

BEGELE = 'BEGINNING'
#ENDELE = 'OTR0H04'
#ENDELE = 'ENDCOL0'
ENDELE = 'END'

# Bmad model

In [5]:
from pytao import Tao
import pandas as pd

In [6]:
tao = Tao(f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init  -slice  {BEGELE}:{ENDELE}')
tao.cmd('place floor bpm_orbit')
tao.cmd('place middle energy')
tao.cmd('sc floor -10 10')

def ele_info(ele):
    dat = tao.ele_head(ele)
    dat.update(tao.ele_gen_attribs(ele))
    return dat

def ele_table(match="*"):
    ix_ele = tao.lat_list(match, "ele.ix_ele", flags="-no_slaves")
    dat = list(map(ele_info, ix_ele))
    df = pd.DataFrame(dat, index=ix_ele)
    df.L.fillna(0, inplace=True)
    df['s_center'] = df['s'] - df['L']/2
    df['s_beginning'] = df['s'] - df['L']
    return  df

df = ele_table()

# Elements with device names
devices = df[df['alias'] != '']
#devices['name alias s'.split()]

In [7]:
tao.cmd('x_scale * 0 800')
print(tao.cmd('sho var'))

['       Name                                      Using for Optimization', '    begtwiss[1:6]', '    gradient_L0[1:8]', '    phi0_L0[1:8]', '    q_HTR[1:8]', '    q_L1[1:5]', '    q_L2[1:4]', '    q_L3[1:4]', '    linac_phase[1:4]', '    q_COL1[1:4]', '    q_EMIT2[1:4]', '    bc1[1:1]', '    bc2[1:1]', '    xcor[1:93]', '    ycor[1:93]']


## Datamaps, and all PVs needed

In [8]:
from copy import deepcopy
def filter_datamap(dm, bmad_names):
    bnames = dm.data['bmad_name'] 
    bmad_names = set(bmad_names)
    ix = bnames[[name in bmad_names for name in bnames]].index
    dm2 = deepcopy(dm)
    dm2.data = dm.data.loc[ix]
    return dm2

In [9]:
DM0 = get_datamaps(MODEL)
DM0.keys()

dict_keys(['bpms', 'cavities', 'correctors', 'tao_energy_measurements', 'quad', 'quad_corrector', 'solenoid'])

In [10]:
good_names = set(df['name'])
bad_eles = [] # any bad eles

for ele in bad_eles:
    good_names.remove(ele)

DM = {}
for name, dm in DM0.items():
    if name == 'tao_energy_measurements':
        # don't filter
        DM[name] = dm
    else:
        DM[name] = filter_datamap(dm, good_names)

DM['cavities'];

In [11]:
# datamaps to exclude
DENYLIST = [
    #'bpms',
   # 'cavities',
    'correctors',
    #'quad',
]

In [20]:
# PVs needed
PVLIST =  []
for name, dm in DM.items():
    if name in DENYLIST:
        continue
    PVLIST.extend(dm.pvlist)
PVLIST = list(set(PVLIST))
len(PVLIST)
PVLIST[0:10]

['BPMS:EMIT2:150:X',
 'ACCL:L2B:0630:AACTMEAN',
 'ACCL:L2B:0770:AACTMEAN',
 'BPMS:HTR:760:X',
 'ACCL:L3B:3530:PACTMEAN',
 'ACCL:L2B:1250:PACTMEAN',
 'ACCL:L2B:1470:PACTMEAN',
 'ACCL:L1B:H220:PACTMEAN',
 'ACCL:L3B:2080:AACTMEAN',
 'ACCL:L3B:1970:AACTMEAN']

In [21]:
os.environ['http_proxy']='socks5h://localhost:8080'
os.environ['HTTPS_PROXY']='socks5h://localhost:8080'
os.environ['ALL_PROXY']='socks5h://localhost:8080'

PVDATA=lcls_archiver_restore(PVLIST, '2023-06-08T23:09:50.000000-07:00')

Requesting: http://lcls-archapp.slac.stanford.edu/retrieval/data/getDataAtTime?at=2023-06-08T23:09:50.000000-07:00&includeProxies=true


In [26]:
cmds=tao_commands(PVDATA)
save_cmds(cmds,filename='live_pvs_bsyd_08-06-23T23:09:50.tao')

## EPICS

In [ ]:
import epics
from epics import caget_many, caget
from time import sleep, time

def caget_dict(pvlist):
    return dict(zip(pvlist, caget_many(pvlist)))

In [ ]:
caget('KLYS:LI22:11:KPHR')

In [ ]:
# Test get
PVDATA = caget_dict(PVLIST)
BAD_DEVICES = set()
PVLIST_GOOD = []
for k, v in PVDATA.items():
    if v is None:
        #print('Bad PV:', k)
        device = ':'.join((k.split(':')[:-1]))
        BAD_DEVICES.add(device)
    else:
        PVLIST_GOOD.append(k)
        
# Get bmad names
bdf = devices[['alias', 'name']].set_index('alias')
BAD_NAMES = list(bdf.loc[list(BAD_DEVICES)]['name'])
#BAD_NAMES        
        
#BAD_DEVICES, BAD_NAMES
print(len(BAD_DEVICES))
print(len(PVLIST_GOOD))

In [ ]:
MONITOR = {pvname:epics.PV(pvname) for pvname in PVLIST_GOOD}
sleep(1) # Wait for all to connect

In [ ]:
def get_pvdata():   
    itime = isotime()
    pvdata =  {k:MONITOR[k].get() for k in MONITOR}
    return pvdata
PVDATA = get_pvdata()

In [ ]:
#print(PVDATA);

# Fiter datamamps again

In [15]:
for name in BAD_NAMES:
    good_names.remove(name)

NameError: name 'BAD_NAMES' is not defined

In [23]:
DM_GOOD = {}
for name, dm in DM.items():
    if name == 'tao_energy_measurements':
        # don't filter
        DM_GOOD[name] = dm
    else:
        DM_GOOD[name] = filter_datamap(dm, good_names)

In [ ]:
print(DM_GOOD['bpms'])

# Tao conveniences

In [17]:
def tao_commands(pvdata):
    cmds = []
    for name, dm in DM_GOOD.items():
        cmds.extend(dm.as_tao(pvdata))
    return cmds

In [18]:
def save_cmds(cmds, filename='cmds.tao'): # Write to file for running with vanilla Tao
    with open(filename, 'w') as f:
        f.write('set global lattice_calc_on = F\n')
        f.write('set global plot_on = F\n')    
        f.write('set ele quad::* field_master = T')
        f.write('set ele SC_L* is_on = F')
        for cmd in cmds:
            f.write(cmd+'\n')
        f.write('set global lattice_calc_on = T\n')        
        f.write('set global plot_on = T\n')   

In [ ]:
# Match HTR to design
def set_htr_twiss(tao):
    cmds="""
vv
vd
set data HTR.begtwiss|meas = HTR.begtwiss|design
use dat HTR.begtwiss[1:4]
use var begtwiss[1:4]
olmdif
""".split('\n') 
    tao.cmds(cmds)
    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('run')
    tao.cmd('set global plot_on = T')
#set_htr_twiss(tao)

In [ ]:
# mat2 x, y for PyEmittance
def get_mats():
    mat6 = tao.matrix('Q0H01#2', 'OTR0H04')['mat6']
    mat2x = mat6[0:2, 0:2]
    mat2y = mat6[2:4, 2:4]
    return mat2x, mat2y

## Form commands using PVDATA and datamaps

In [ ]:
# Master switches for element scaling
tao.cmd('set ele quad::* field_master = T')

# Turn off phase overlays 
tao.cmd('set ele SC_L* is_on = F', raises=False)

if MODEL == 'sc_inj':
    tao.cmd('set ele lcavity::* autoscale_phase = T')

In [ ]:
pvdata = get_pvdata()
cmds=tao_commands(pvdata)
save_cmds(cmds,filename='live_pvs_26_05_23_1500.tao')

In [ ]:
tao.cmd('scale top 0 5000')

# Continuous loop

In [ ]:
def run1():
    #sleep(.001)
    t1 = time()
    pvdata = get_pvdata()
    cmds = tao_commands(pvdata)
    tao.cmd('set global plot_on = F;set global lattice_calc_on = F')
    tao.cmds(cmds); # Apply

    tao.cmd('set global lattice_calc_on = T')
    tao.cmd('set global plot_on = T')
    #toggle_beam()
    
    dt = time()-t1
    #print(dt)    


In [ ]:
tao.cmd('set var *|model=*|design')
tao.cmd('set global lattice_calc_on=T')

In [ ]:
%%time
run1()

In [ ]:
# Set twiss
set_htr_twiss(tao)

In [ ]:
# Check charge (pC)
df = pd.DataFrame(tao.data_d_array('orbit', 'charge'))
df['charge_live'] =df['meas_value']*1e12
df[['ele_name', 'charge_live']]

In [ ]:
tao.cmd('set var *|model=*|design')

In [ ]:
%%tao
use dat orbit.x
use dat orbit.y
sc floor -5 5
sc top
x_scale * 0 50
sc top

# Continuous run
    

In [ ]:
# Run forever
while True:
    run1()